In [1]:
import os
import re
import pandas as pd
from collections import defaultdict
from datetime import date

## Load Raw Data

In [2]:
raw_data_folder = '../02.raw-data/'
output_data_folder = '../03.clean-data/'

In [ ]:
if not os.path.isdir(output_data_folder):
    if not os.path.exists(output_data_folder):
        os.mkdir(output_data_folder)
    else:
        raise NotADirectoryError
else:
    return True

In [3]:
def remove_extra_characters_at_the_end(expression: str, startswith: str) -> str:
    if expression.startswith(startswith):
        return startswith
    else:
        return expression

In [4]:
def clean_area_name(area: str) -> str:
    area = remove_extra_characters_at_the_end(area, 'York Region')
    area = remove_extra_characters_at_the_end(area, 'Clarington')
    area = remove_extra_characters_at_the_end(area, 'Georgina')
    area = remove_extra_characters_at_the_end(area, 'Innisfil')
    area = remove_extra_characters_at_the_end(area, 'Milton')
    area = remove_extra_characters_at_the_end(area, 'Orangeville')
    if area.startswith('Toronto'):
        m = re.match('(Toronto\s+[A-Z])([0-9ODZS]{2})', area)
        if m:
            beginning = m.group(1)
            two_number_code_at_the_end = m.group(2)
            two_number_code_at_the_end = (
                two_number_code_at_the_end
                    .replace('O', '0')
                    .replace('D', '0')
                    .replace('S', '5')
                    .replace('Z', '2'))
        area = beginning + two_number_code_at_the_end
    return area

In [5]:
raw_filenames = os.listdir(raw_data_folder)
dfs = []
for filename in raw_filenames:
    if filename.startswith('all-trreb-areas') or filename.startswith('city-of-toronto'):
        label, year, month = re.search('([a-z\-]+)-([0-9]{2})-([0-9]{2})\.csv', filename).groups()
        df = pd.read_csv(os.path.join(raw_data_folder, filename))
        df['Date'] = date(2000 + int(year), int(month), 1)
        df['Area'] = df['Area'].map(clean_area_name)
        df = df.melt(id_vars=['Area', 'Date'],
                     value_vars=list(set(df.columns) - set(['Area', 'Date'])),
                     var_name='Type',
                     value_name='HPI')
        df['HPI'] = (df['HPI']
            .astype(str)
            .replace('([0-9]{3})./', r'\1.7', regex=True)
            .replace('(25%3);3)', '255.9')
            .replace('(23\8).:3)', '299.9')
            .replace('^-$', 'NaN', regex=True)
            .replace('Zion', '275.5')
            .replace('Za', '295.9')
            .replace('([0-9]{3})°([0-9])', r'\1.\2', regex=True)
            .astype(float)
        )
        dfs.append(df)
all_areas = pd.concat(dfs).reset_index()

## Find Outliers

In [6]:
all_areas['HPI'].describe()

count    28724.000000
mean       229.250226
std        293.514708
min          0.000000
25%        182.000000
50%        230.600000
75%        262.600000
max      29519.000000
Name: HPI, dtype: float64

In [7]:
large_hpi_values = all_areas['HPI'].sort_values(ascending=False).head(20)
all_areas[all_areas['HPI'].isin(large_hpi_values)]

,index,Area,Date,Type,HPI
10856,151,Brock,2020-05-01,Single-Family Attached,604.1
11016,146,Brock,2020-06-01,Single-Family Attached,600.7
11177,147,Brock,2020-07-01,Single-Family Attached,643.8
11342,152,Brock,2020-08-01,Single-Family Attached,671.3
11497,142,Brock,2020-09-01,Single-Family Attached,670.5
11656,146,Brock,2020-10-01,Single-Family Attached,709.0
12560,90,Oshawa,2021-04-01,Townhouse,441.6
28957,47,Toronto C03,2020-02-01,Single-Family Detached,29519.0
28994,84,Toronto C06,2020-02-01,Townhouse,25959.0
29383,123,Toronto C10,2020-04-01,Apartment,3295.2


## Manual Corrections

In [8]:
# Manual corrections
def update_cell(dt, area, home_type, hpi):
    row_index = all_areas[(all_areas['Date'] == dt) & (all_areas['Area'] == area) & (all_areas['Type'] == home_type)]['HPI'].index
    all_areas.loc[row_index, 'HPI'] = hpi

update_cell(date(2020, 4, 1), 'Toronto C10', 'Apartment',              326.6)
update_cell(date(2020, 2, 1), 'Toronto C06', 'Townhouse',              259.5)
update_cell(date(2020, 2, 1), 'Toronto C03', 'Single-Family Detached', 295.5)
update_cell(date(2021, 1, 1), 'Toronto E05', 'Townhouse',              295.5)


In [9]:
all_areas = all_areas[~((all_areas['Area'] == "Brock") & (all_areas['Type'] == "Single-Family Attached"))]

In [10]:
all_areas['HPI'].describe()

count    28712.000000
mean       226.108157
std         54.703810
min          0.000000
25%        182.000000
50%        230.600000
75%        262.600000
max        479.800000
Name: HPI, dtype: float64

In [11]:
all_areas[all_areas['HPI'].isin(all_areas['HPI'].sort_values(ascending=False).head(10))]

,index,Area,Date,Type,HPI
12560,90,Oshawa,2021-04-01,Townhouse,441.6
29879,94,Toronto E01,2020-07-01,Townhouse,437.6
30054,94,Toronto E01,2020-08-01,Townhouse,434.5
30229,94,Toronto E01,2020-09-01,Townhouse,436.9
30404,94,Toronto E01,2020-10-01,Townhouse,434.4
30579,94,Toronto E01,2020-11-01,Townhouse,433.6
30929,94,Toronto E01,2021-01-01,Townhouse,437.8
31104,94,Toronto E01,2021-02-01,Townhouse,450.1
31279,94,Toronto E01,2021-03-01,Townhouse,466.8
31454,94,Toronto E01,2021-04-01,Townhouse,479.8


## Drop areas with missing data

In [12]:
city_of_toronto_data_counts = all_areas[all_areas['Area'].str.startswith('Toronto')].groupby('Area').apply(len)
all_trreb_areas_data_counts = all_areas[~all_areas['Area'].str.startswith('Toronto')].groupby('Area').apply(len)

city_of_toronto_areas_to_keep = city_of_toronto_data_counts[city_of_toronto_data_counts == city_of_toronto_data_counts.median()].index.tolist()
all_trreb_areas_to_keep = all_trreb_areas_data_counts[all_trreb_areas_data_counts == all_trreb_areas_data_counts.median()].index.tolist()


In [13]:
all_trreb_areas_data_counts

Area
Adjala-Tosorontio              345
Ajax                           345
Aurora                         345
Bradford West                  265
Bradford West Gwillimbury       80
Brampton                       345
Brock                          276
Burlington                     345
Caledon                        345
City of Toronto                700
Clarington                     345
Dufferin                         5
Dufferin County                300
Durham Region                  275
EGswsiallimbury                140
East Gwillimbury               345
Essa                            80
GEswsiallimbury                125
Georgina                       345
Halton Hills                   345
Halton Region                  265
Innisfil                       345
King                           345
Markham                        345
Meee Le er a                     5
Milton                         345
Mississauga                    345
New Tecumseth                  345
Newmarket      

In [14]:
city_of_toronto_data_counts

Area
Toronto C01    515
Toronto C02    515
Toronto C03    515
Toronto C04    515
Toronto C06    515
Toronto C07    515
Toronto C08    515
Toronto C09    515
Toronto C10    515
Toronto C11    515
Toronto C12    515
Toronto C13    515
Toronto C14    515
Toronto C15    515
Toronto E01    515
Toronto E02    515
Toronto E03    515
Toronto E04    515
Toronto E05    515
Toronto E06    515
Toronto E07    515
Toronto E08    515
Toronto E09    515
Toronto E10    515
Toronto E11    515
Toronto W01    515
Toronto W02    515
Toronto W03    515
Toronto W04    515
Toronto W05    515
Toronto W06    515
Toronto W07    515
Toronto W08    515
Toronto W09    515
Toronto W10    515
dtype: int64

In [15]:
city_of_toronto_housing_types = all_areas[all_areas['Area'].str.startswith('Toronto')].groupby('Area')['Type'].nunique()
all_trreb_areas_housing_types = all_areas[~all_areas['Area'].str.startswith('Toronto')].groupby('Area')['Type'].nunique()

all_trreb_areas_housing_types

Area
Adjala-Tosorontio              5
Ajax                           5
Aurora                         5
Bradford West                  5
Bradford West Gwillimbury      5
Brampton                       5
Brock                          4
Burlington                     5
Caledon                        5
City of Toronto                5
Clarington                     5
Dufferin                       5
Dufferin County                5
Durham Region                  5
EGswsiallimbury                5
East Gwillimbury               5
Essa                           5
GEswsiallimbury                5
Georgina                       5
Halton Hills                   5
Halton Region                  5
Innisfil                       5
King                           5
Markham                        5
Meee Le er a                   5
Milton                         5
Mississauga                    5
New Tecumseth                  5
Newmarket                      5
Oakville                       5
Orang

In [16]:
clean_dataset = all_areas[all_areas['Area'].isin(city_of_toronto_areas_to_keep + all_trreb_areas_to_keep + ['TREB Total'])]
clean_dataset.to_csv(os.path.join(output_data_folder, 'hpi.csv'), index=False)

In [17]:
len(clean_dataset)

28040

In [18]:
with pd.option_context('max_rows', None):
    display(clean_dataset[(clean_dataset['Date'] == date(2020, 4, 1)) & (clean_dataset['Area'] == 'Toronto C10') & (clean_dataset['Type'] == 'Apartment')])

,index,Area,Date,Type,HPI
29383,123,Toronto C10,2020-04-01,Apartment,326.6


In [19]:
clean_dataset['HPI'].sort_values(ascending=False).head(20)

31454    479.8
31279    466.8
31104    450.1
12560    441.6
30929    437.8
29879    437.6
30229    436.9
30054    434.5
30404    434.4
30579    433.6
12395    432.7
31524    430.9
29179    430.5
29704    426.7
29354    426.2
30754    425.1
29529    424.5
31499    423.2
31419    421.6
31384    420.5
Name: HPI, dtype: float64

In [20]:
clean_dataset.dtypes

index      int64
Area      object
Date      object
Type      object
HPI      float64
dtype: object

In [21]:
clean_dataset.sample(25)

,index,Area,Date,Type,HPI
11331,141,Aurora,2020-08-01,Single-Family Attached,289.6
6757,107,Orangeville,2018-06-01,Townhouse,NaN
3456,36,Innisfil,2017-01-01,Composite,216.2
772,12,Aurora,2015-11-01,Composite,200.1
9616,116,Burlington,2019-10-01,Apartment,269.9
4535,165,East Gwillimbury,2017-06-01,Single-Family Attached,283.4
23396,26,Toronto E01,2017-07-01,Composite,307.0
4104,114,TREB Total,2017-04-01,Apartment,224.3
6953,113,New Tecumseth,2018-07-01,Townhouse,NaN
15903,118,Toronto W06,2014-02-01,Apartment,120.7
